# BeerAdvocate

In [1]:
import pandas as pd

## Files loading

In [3]:
beers_adv = pd.read_csv("../../data/BeerAdvocate/beers.csv", sep=",")
breweries_adv = pd.read_csv("../../data/BeerAdvocate/breweries.csv", sep=",")
users_adv = pd.read_csv("../../data/BeerAdvocate/users.csv", sep=",")